<a href="https://colab.research.google.com/github/shikhar00778/toxic-comment-classification/blob/master/Bert_220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/12500/363832/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1559217188&Signature=WxgmexT%2FEzJj7SskfbmJohBGA9bbqMjz1wXJs%2BaoQvToWKmJAtbrCb4E1dE90Fm00PzRrXMN9tdIhaaRyyQrh650xv5Sjeiq%2BK3hR%2BJ2d3LHkp7Y2H9HVp8e2B9SQauyVgBXltiIc41Fkwp%2FkRccnpfhydbhnmymGhfoH6cCnomO9%2BNuRdBX7G6QjKoymmEzyXPYJyMXNMbVBvmaw%2BB4X%2F4MmZe%2FSecjCxVSz03drupVml6fE0ZUzSLbvxSEWLwP0hMzfWzyH9iMrEQkifMGyyBjHMGwnActYcjbqdOKH4U7HSUw1XDC%2FGrfp4SDhgVKGBBsiqFy7gRo8GXAH0HQXA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-unintended-bias-in-toxicity-classification.zip" -O "jigsaw-unintended-bias-in-toxicity-classification.zip" -c

--2019-05-29 15:34:32--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/12500/363832/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1559217188&Signature=WxgmexT%2FEzJj7SskfbmJohBGA9bbqMjz1wXJs%2BaoQvToWKmJAtbrCb4E1dE90Fm00PzRrXMN9tdIhaaRyyQrh650xv5Sjeiq%2BK3hR%2BJ2d3LHkp7Y2H9HVp8e2B9SQauyVgBXltiIc41Fkwp%2FkRccnpfhydbhnmymGhfoH6cCnomO9%2BNuRdBX7G6QjKoymmEzyXPYJyMXNMbVBvmaw%2BB4X%2F4MmZe%2FSecjCxVSz03drupVml6fE0ZUzSLbvxSEWLwP0hMzfWzyH9iMrEQkifMGyyBjHMGwnActYcjbqdOKH4U7HSUw1XDC%2FGrfp4SDhgVKGBBsiqFy7gRo8GXAH0HQXA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-unintended-bias-in-toxicity-classification.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299148730 (285M) [application/zip]
Saving to: ‘jigsaw-unintended-bias-in

In [2]:
!unzip jigsaw-unintended-bias-in-toxicity-classification.zip

Archive:  jigsaw-unintended-bias-in-toxicity-classification.zip
  inflating: train.csv               
  inflating: sample_submission.csv   
  inflating: test.csv                


In [3]:
!pip install git+https://github.com/kaushaltrivedi/fast-bert.git

  Cloning https://github.com/kaushaltrivedi/fast-bert.git to /tmp/pip-req-build-lqsxz443
  Running command git clone -q https://github.com/kaushaltrivedi/fast-bert.git /tmp/pip-req-build-lqsxz443
     |████████████████████████████████| 133kB 2.8MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-t7f85iy5/wheels/53/11/ef/67d507c8f4f05b843a6dd2c4f6360acdee4c79679c4ca36bc6
Successfully built fast-bert


In [4]:
%%writefile /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py
import numpy as np
from torch import Tensor
from sklearn.metrics import roc_curve, auc, hamming_loss, accuracy_score
import pdb

# def accuracy(out, labels):
#     outputs = np.argmax(out, axis=1)
#     return np.sum(outputs == labels)

def accuracy(y_pred:Tensor, y_true:Tensor):

    outputs = np.argmax(y_pred, axis=1)
    return np.mean(outputs.numpy() == y_true.detach().cpu().numpy())

def accuracy_multilabel(y_pred:Tensor, y_true:Tensor, sigmoid:bool=True):
    if sigmoid: y_pred = y_pred.sigmoid()
    outputs = np.argmax(y_pred, axis=1)
    real_vals = np.argmax(y_true, axis=1)
    return np.mean(outputs.numpy() == real_vals.numpy())

def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.byte()).float().mean().item()
#     return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()


def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.3, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    "Computes the f_beta between `preds` and `targets`"
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean().item()

def roc_auc(y_pred: Tensor, y_true: Tensor):
    # ROC-AUC calcualation
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    return roc_auc["micro"]


Overwriting /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py


In [5]:
!pip uninstall apex

In [0]:
!rm -rf apex

In [7]:
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!python setup.py install --cuda_ext --cpp_ext

Cloning into 'apex'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 4530 (delta 84), reused 86 (delta 58), pack-reused 4394
Receiving objects: 100% (4530/4530), 8.69 MiB | 23.00 MiB/s, done.
Resolving deltas: 100% (2912/2912), done.
/content/apex
torch.__version__  =  1.1.0

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
writing manifest file 'apex.egg-info/SOURCES.txt'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install

In [8]:
cd ..

/content


In [4]:
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import text,sequence
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.metrics import *
import os
import random

import apex
import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertModel
from pytorch_pretrained_bert.tokenization import BertTokenizer

from fast_bert.data import BertDataBunch
from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy , accuracy_multilabel , roc_auc

Using TensorFlow backend.


In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir=None,do_lower_case=True)

In [0]:
device = torch.device('cuda')

# check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [0]:
import pandas as pd
df = pd.read_csv('train.csv')

In [0]:
df_valid = df[1700000:]

In [0]:
df_train = df[:700000]

In [0]:
df_train = df_train.rename(index=str, columns={"comment_text": "text"})

In [0]:
df_valid = df_valid.rename(index=str, columns={"comment_text": "text"})

In [0]:

df_train.loc[df_train['target'] >= 0.5 , ['target']] = 1
df_train.loc[df_train['target'] < 0.5 , ['target']] = 0


df_valid.loc[df_valid['target'] >= 0.5 , ['target']] = 1
df_valid.loc[df_valid['target'] < 0.5 , ['target']] = 0

In [0]:
df_train.loc[df_train['severe_toxicity'] >= 0.5 , ['severe_toxicity']] = 1
df_train.loc[df_train['severe_toxicity'] < 0.5 , ['severe_toxicity']] = 0


df_valid.loc[df_valid['severe_toxicity'] >= 0.5 , ['severe_toxicity']] = 1
df_valid.loc[df_valid['severe_toxicity'] < 0.5 , ['severe_toxicity']] = 0



In [0]:
df_train.loc[df_train['obscene'] >= 0.5 , ['obscene']] = 1
df_train.loc[df_train['obscene'] < 0.5 , ['obscene']] = 0


df_valid.loc[df_valid['obscene'] >= 0.5 , ['obscene']] = 1
df_valid.loc[df_valid['obscene'] < 0.5 , ['obscene']] = 0


In [0]:
df_train.loc[df_train['identity_attack'] >= 0.5 , ['identity_attack']] = 1
df_train.loc[df_train['identity_attack'] < 0.5 , ['identity_attack']] = 0


df_valid.loc[df_valid['identity_attack'] >= 0.5 , ['identity_attack']] = 1
df_valid.loc[df_valid['identity_attack'] < 0.5 , ['identity_attack']] = 0

In [0]:
df_train.loc[df_train['insult'] >= 0.5 , ['insult']] = 1
df_train.loc[df_train['insult'] < 0.5 , ['insult']] = 0


df_valid.loc[df_valid['insult'] >= 0.5 , ['insult']] = 1
df_valid.loc[df_valid['insult'] < 0.5 , ['insult']] = 0

In [0]:
df_train.loc[df_train['threat'] >= 0.5 , ['threat']] = 1
df_train.loc[df_train['threat'] < 0.5 , ['threat']] = 0


df_valid.loc[df_valid['threat'] >= 0.5 , ['threat']] = 1
df_valid.loc[df_valid['threat'] < 0.5 , ['threat']] = 0

In [0]:
!rm -r train_sample.csv

In [0]:
!rm -r valid.csv

In [0]:
df_train.to_csv('train_sample.csv' , index = False)

In [0]:
df_valid.to_csv('valid.csv' , index = False)

In [25]:
!ls

apex						       test.csv
jigsaw-unintended-bias-in-toxicity-classification.zip  train.csv
labels.csv					       train_sample.csv
sample_data					       valid.csv
sample_submission.csv


In [0]:
l = ['target','severe_toxicity', 'obscene','identity_attack', 'insult', 'threat']
df_label = pd.DataFrame(l)

df_label.to_csv('labels.csv' , index = False , header = None)

In [0]:

args = {
    "max_seq_length": 220,
    "do_lower_case": True,
    "train_batch_size": 32,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

In [0]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
databunch = BertDataBunch('/content', '/content', tokenizer, 
                          train_file='train_sample.csv', val_file='valid.csv',label_file='labels.csv',
                          label_col=['target','severe_toxicity', 'obscene','identity_attack', 'insult', 'threat'],
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)

In [10]:
metrics = []
metrics.append({'name': 'roc_auc', 'function': roc_auc})

learner = BertLearner.from_pretrained_model(databunch, 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz', metrics, device, logger=logger, 
                                            finetuned_wgts_path=None, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)

05/29/2019 19:33:45 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmpk6e_oje4
100%|██████████| 407873900/407873900 [00:05<00:00, 71998817.64B/s]
05/29/2019 19:33:50 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpk6e_oje4 to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
05/29/2019 19:33:53 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
05/29/2019 19:33:53 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmpk6e_oje4
05/29/2019 19:33:53 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.co

In [11]:
learner.fit(1, lr=args['learning_rate'], 
            schedule_type="warmup_cosine_hard_restarts")

05/29/2019 22:33:55 - INFO - __main__ -   Loss after epoch 0 - 0.04695184600830078
05/29/2019 22:33:55 - INFO - __main__ -   Running evaluation


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/ranking.py:659: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
05/29/2019 22:53:54 - INFO - __main__ -   Eval results:
05/29/2019 22:53:54 - INFO - __main__ -     eval_loss = 0.041112106845755635
05/29/2019 22:53:54 - INFO - __main__ -     metrics = {'roc_auc': 0.9863789144337757}
05/29/2019 22:53:54 - INFO - __main__ -   --------------------------------------------------------------------------------


In [0]:
df_test = pd.read_csv('test.csv')

In [0]:
texts = list(df_test['comment_text'])

In [14]:
target = []

import tqdm
for i in tqdm.tqdm(range(0,len(texts),24)):
  txt = texts[i:i+24]
  predictions = learner.predict_batch(txt)
  
  for j in range(24):
    target.append(predictions[j][1][1])
  #break

100%|██████████| 4055/4055 [10:43<00:00,  6.28it/s]


In [0]:
df_test['target'] = target

In [0]:
df_test = df_test.drop(['comment_text'] , axis = 1)

In [0]:
df_test.to_csv('submission_bert_jigsaw_220_1m.csv' , index = False)

In [0]:
from google.colab import files
files.download('submission_bert_jigsaw_220_1m.csv')